In [277]:
import pandas as pd
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation

In [278]:
df = spark.read.csv("house_prices.csv", inferSchema = True, header = True)

In [279]:

idColumnName = 'Id'
salePriceColumnName = 'SalePrice'
maxCorrelation = -1
maxCorrelationColumnName = ""

for item in df.dtypes:
    if not item[1].startswith('string') and item[0] != idColumnName and item[0] != salePriceColumnName:
        correlation = df.stat.corr(item[0], salePriceColumnName, "pearson")
        if correlation > maxCorrelation:
            maxCorrelation = correlation
            maxCorrelationColumnName = item[0]

print(maxCorrelationColumnName, maxCorrelation)

OverallQual 0.7909816005838053


In [280]:
def columnsFilter(column):
    notString = not column[1].startswith('string')
    notId = column[0] != idColumnName
    notSalePrice = column[0] != salePriceColumnName
    return notString and notId and notSalePrice

def correlationMap(column):
    correlation = df.stat.corr(column[0], salePriceColumnName, "pearson")
    return (column[0], correlation)

filteredColumns = list(filter(columnsFilter, df.dtypes))
correlationColumns = map(correlationMap, filteredColumns)
print(max(correlationColumns, key=lambda item:item[1]))



('OverallQual', 0.7909816005838053)
